## CLUSTERING:

### IMPORT MODULES:

In [1]:
import os
import re
import csv
import nltk
import mpld3
import codecs
import collections

import math as mh
import numpy as np
import pandas as pd
import string as st
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt

from time import time
from itertools import chain
from itertools import islice
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import feature_extraction
from sklearn.cluster import MiniBatchKMeans
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/deniz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/deniz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### READ CSVS:

In [2]:
with open("../data/vkusvill_items.csv", 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    catalog_df = pd.DataFrame(reader)

catalog_df = catalog_df.rename(columns = {"": "shop_id", "item_composition_txt": "item_composition", "nutrion_value_txt" : "nutrion_value"})
catalog = catalog_df.drop(["shop_id", "price", "vat", "measure_unit", "measure_value", "protein_value", "fat_value", "carb_value", 
                           "energy_value", "measure_quantum"], axis=1)

In [3]:
categories_df = pd.read_csv("../data/vkusvill_categories.csv")
categories_df = categories_df.rename( columns = {"Unnamed: 0": "shop_id"})

### PREPARE DATA:

In [4]:
def check_categorie(df, categorie):
    result = pd.DataFrame();
    
    for line in range(len(df.index)):
        if str(categorie) in df["categories_array"][line]:
            df_string = pd.DataFrame({"item_id": [df["item_id"][line]], "item_name": [df["item_name"][line]]})
            
            result = pd.concat([result, df_string], ignore_index=True)

    return result
    
result = check_categorie(catalog, 13152)  # Фарш

In [5]:
result

,item_id,item_name
0,43499,Фарш из кролика и индейки
1,61936,"Фарш из мяса курицы, индейки и кролика"
2,62798,"Фарш из говядины \Постный по стандарту ВВ\, 360 г"
3,62804,"Фарш из телятины, 360 г"
4,62806,"Фарш из говядины и свинины, 360 г"
5,62809,"Фарш из говядины \по стандарту ВкусВилл\, 360 г"
6,69576,"Фарш \Домашний по стандарту ВкусВилл\, 360 г"
7,71029,Фарш из говядины и курицы
8,20336,"Фарш из кролика, 500 г"
9,21194,"Фарш куриный охлажденный, 500 г"


### PARS ITEM NAMES:

In [6]:
# Add more

replace_items = {
                # Предлоги
                ' в ': ' ', ' к ': ' ', " до ": ' ', " по ": ' ', " через ": ' ', " после ": ' ', " в течение ": ' ', 
                 " в продолжение ": ' ', " в заключение ": ' ', " из-за ": ' ', " над ": ' ', " под ": ' ', " перед ": ' ', 
                ' у ': ' ', " возле ": ' ', " мимо ": ' ', " около ": ' ', " от ": ' ', " ради ": ' ', " благодаря ": ' ', 
                " в силу ": ' ', " ввиду ": ' ', " вследствие ": ' ', " для ": ' ',  " на ": ' ', " в целях ": ' ', 
                " с целью ": ' ', " вопреки ": ' ', " несмотря на ": ' ', ' о ': ' ', " об ": ' ', " обо ": ' ', 
                " про ": ' ', " насчёт ": ' ', ' с ': ' ', " вроде ": ' ', " наподобие ": ' ', " как ": ' ', " без ": ' ',
                # Союзы
                 " что ": ' ', " когда ": ' ', " ибо ": ' ', " пока ": ' ', " будто ": ' ', " словно ": ' ', " если ": ' ', " кто ": ' ', 
                 " который ": ' ', " какой ": ' ', " где ": ' ', " куда ": ' ', " откуда ": ' ', " потому что ": ' ', " оттого что ": ' ', 
                 " так как ": ' ', " так что ": ' ', " лишь только ": ' ', " как будто ": ' ', " с тех пор как ": ' ', 
                 " в связи с тем что ": ' ', " для того чтобы ": ' ', ' и ': ' ', " да ": ' ', " ни ни ": ' ', " тоже ": ' ', 
                 " также ": ' ', " не только ": ' ', " не столько ": ' ', " не то чтобы ": ' ', " или ": ' ', " лишь ": ' ', 
                 " либо ": ' ', " то ": ' ', " не то ": ' ', " а ": ' ', " но ": ' ', " зато ": ' ', " однако ": ' ', " то есть ": ' ', 
                 " а именно ": ' ', " чтоб ": ' ', " чтобы ": ' ', " с тех пор ": ' ', " едва ": ' ', " прежде чем ": ' ', 
                 " перед тем как ": ' ', " точно ": ' ', " подобно ": ' ', " дабы ": ' ', " коли ": ' ', " ежели ": ' ', " раз ": ' ', 
                 " пускай ": ' ', " хотя ": ' ', " поэтому ": ' ',
                 # Другие слова
                 " шт ": ' ', " мл ": ' ', " мг ": ' ', " премиум ": ' ', " вес ": ' ', " из ": ' ', " спб ": ' ',
                 " вкусвилл ": ' ', " стандарту ": ' ', " вв ": ' ', " ве ": ' ', ' г ': ' '
                 }

In [7]:
def replace_all(text, replace_items):
    for i, j in replace_items.items():
        text = text.lower().replace(i, j)
        
    return text

In [8]:
def clear_item_name(item_name, replace_items):
    answer = ""
    
    for symbol in item_name:
        if symbol == '_':
            answer += ' '  
        elif symbol in [
            '\\', 
            '(', 
            ')', 
            '.', 
            ',', 
            '%'
        ] or symbol.isdigit():
            answer += ' '
        else:
            answer += symbol

    answer = answer.lower() + ' '
    
    while answer != replace_all(answer, replace_items):
        answer = replace_all(answer, replace_items)
    
    while "  " in answer:
        answer = answer.replace("  ", ' ')

    return answer.lower()

In [9]:
prepare_catalog_df = pd.Series()

for i in range(len(result.index)):
    tmp = result["item_name"][i]
    
    while tmp != clear_item_name(result["item_name"][i], replace_items):
        tmp = clear_item_name(result["item_name"][i], replace_items)
    
    prepare_catalog_df = pd.concat([prepare_catalog_df, pd.Series(tmp[:-1])], ignore_index=True)

In [10]:
prepare_catalog_df

0                 фарш кролика индейки
1     фарш мяса курицы индейки кролика
2                фарш говядины постный
3                        фарш телятины
4                фарш говядины свинины
5                        фарш говядины
6                        фарш домашний
7                 фарш говядины курицы
8                         фарш кролика
9             фарш куриный охлажденный
10                        фарш куриный
11                        фарш индейки
12                         фарш свиной
13                       фарш домашний
14                       фарш говядины
15                        фарш индейки
16                        фарш куриный
17                         фарш свиной
18                       фарш телятины
19                       фарш домашний
20                   фарш филе индейки
21                  фарш филе куриного
22                        фарш говяжий
23                         фарш свиной
dtype: object

In [11]:
final_result = pd.concat([result, prepare_catalog_df], ignore_index=True, axis=1)
final_result = final_result.rename( columns = {0: "item_id", 1: "item_name", 2: "clear_item_name"})
# final_result = final_result.drop("item_name", axis=1)

In [12]:
final_result

,item_id,item_name,clear_item_name
0,43499,Фарш из кролика и индейки,фарш кролика индейки
1,61936,"Фарш из мяса курицы, индейки и кролика",фарш мяса курицы индейки кролика
2,62798,"Фарш из говядины \Постный по стандарту ВВ\, 360 г",фарш говядины постный
3,62804,"Фарш из телятины, 360 г",фарш телятины
4,62806,"Фарш из говядины и свинины, 360 г",фарш говядины свинины
5,62809,"Фарш из говядины \по стандарту ВкусВилл\, 360 г",фарш говядины
6,69576,"Фарш \Домашний по стандарту ВкусВилл\, 360 г",фарш домашний
7,71029,Фарш из говядины и курицы,фарш говядины курицы
8,20336,"Фарш из кролика, 500 г",фарш кролика
9,21194,"Фарш куриный охлажденный, 500 г",фарш куриный охлажденный


### CLUSTERING:

In [13]:
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    
    stems = [stemmer.stem(token) for token in filtered_tokens]
    return stems

def token_only(text):
    tokens = [word.lower() for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)]
    filtered_tokens = []
    
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    
    return filtered_tokens

# Создаем словари из полученных основ
total_vocab_stem = []
total_vocab_token = []

for i in range(len(result.index)):
    all_words_stemmed = token_and_stem(final_result["clear_item_name"][i])
    # print(all_words_stemmed)
    total_vocab_stem.extend(all_words_stemmed)    
    all_words_tokenized = token_only(final_result["clear_item_name"][i])
    total_vocab_token.extend(all_words_tokenized)

In [14]:
print('Товаров из категории считано: ' + str(len(final_result)))

Товаров из категории считано: 24


In [15]:
stopwords = nltk.corpus.stopwords.words('russian')
#можно расширить список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', "из", "по", 'г', "вв", "вкуссвилл", "вес", "спб"])

# ["были" том пожалуйста около вон день недалеко самими сам мож кто много восемнадцатый к же четыре ваше близко им несколько мира ту по, "быть", 
# "которой", "тринадцать "потом "никуда "свое "говорит "одиннадцать ваша "можно "этом "туда", "само", "говорил твоя будешь разве наверху 
# этой ней тысяч твоё саму двенадцатый всё нем перед время весь обычно позже второй ещё пятый чего имя вокруг другая чаще сказала та самих
# многочисленная раньше отсюда всею самим моё между тобой вдруг у есть слишком про нет эту какая пять всего тринадцатый свои их 
# когда ними мои более меля нередко нас себя совсем сих этих меньше уж сказал жизнь кроме самой всю её кем ты и одиннадцатый три
# ни суть двадцатый десятый году пятнадцатый такие будете которых самого чтобы нам часто чтоб конечно два зачем всему многочисленные
# будут их они наше пора нибудь заняты оба могут всюду очень каждая лишь рядом твой только это все шестой люди мой самом ж внизу девятый
# себе меня почему его мимо вверх будет всегда как об которого вот собой то уже в мор мочь сказать эти снова многочисленное человек
# сколько своих она занята важный мы вы восемь чем неё двадцать дальше ком них семнадцать лучше отовсюду всем седьмой других он который
# сегодня надо такое оно больше моя нею вниз вами другое шестнадцатый сами мог теми такая нужно иметь этого многочисленный после однажды
# теперь наиболее занято времени
# самому года посреди потому шесть тоже начала тех хотя один ниже каждый может каждые сейчас собою девять мало наши важные 
# где долго также которая тебе было чуть куда рано занят действительно для вам впрочем что всеми четырнадцать без нельзя против 
# а тебя почти прекрасно вся семнадцатый друго мною
# своего сначала процентов стал даже дел те эта кому миллионов затем вас ли значит давно первый пор ее ваши раз тут какой недавно уметь сеаой 
# немного наконец именно довольно ним т зато хотеть важная до иногда ему тот две кругом мне которые просто или там пока менее бывь опять десять
# вдали хоть ими лет другой восьмой хочешь еще наша ведь тому через был здесь сама во этим двенадцать бывает нему этими него
# буду кажется была пятнадцать не с можхо наш ей спасибо другие тобою хорошо двух из
# четырнадцатый этот одного низко важное даром будь ну ею
# "однако", "будем", "мной", "должно", "тою", "нами", "девятнадцатый", "ваш на
# нее всех алло тем далеко своей так особенно восемнадцать ничего над четвертый
# кого шестнадцать год такой чему под если будто того одной свою везде при но со", 'о', "от", "да", "семь", 'г', "за", "никогда", "непрерывно", 
# "этому", "каждое", "третий", "девятнадцать", "назад", "тогда", "бы"]

n_featur = 2000000

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))

get_ipython().magic('time tfidf_matrix = tfidf_vectorizer.fit_transform(final_result["clear_item_name"])')
print(tfidf_matrix.shape)

CPU times: user 29.2 ms, sys: 390 µs, total: 29.5 ms
Wall time: 29.4 ms
(24, 44)


/tmp/ipykernel_34186/54009743.py:35: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time tfidf_matrix = tfidf_vectorizer.fit_transform(final_result["clear_item_name"])')
/home/deniz/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/deniz/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 

In [16]:
num_clusters = 20

# Метод к-средних - KMeans
km = KMeans(n_clusters=num_clusters)
get_ipython().magic('time km.fit(tfidf_matrix)')
idx = km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

# MiniBatchKMeans
mbk  = MiniBatchKMeans(init='random', n_clusters=num_clusters) #(init='k-means++', ‘random’ or an ndarray)
mbk.fit_transform(tfidf_matrix)
%time mbk.fit(tfidf_matrix)
miniclusters = mbk.labels_.tolist()
# print (mbk.labels_)

# # DBSCAN
get_ipython().magic('time db = DBSCAN(eps=0.3, min_samples=10).fit(tfidf_matrix)')
labels = db.labels_
labels.shape
# print(labels)

# # Аггломеративная класстеризация
agglo1 = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean') #affinity можно выбрать любое или попробовать все по очереди: cosine, l1, l2, manhattan
get_ipython().magic('time answer = agglo1.fit_predict(tfidf_matrix.toarray())')
answer.shape

/tmp/ipykernel_34186/629820274.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time km.fit(tfidf_matrix)')
/home/deniz/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/deniz/.local/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (16) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/deniz/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs

CPU times: user 332 ms, sys: 0 ns, total: 332 ms
Wall time: 291 ms
CPU times: user 10.4 ms, sys: 667 µs, total: 11.1 ms
Wall time: 5.14 ms
CPU times: user 1.45 ms, sys: 0 ns, total: 1.45 ms
Wall time: 1.71 ms
CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 4.73 ms


/home/deniz/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/home/deniz/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
/tmp/ipykernel_34186/629820274.py:17: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('time db = DBSCAN(eps=0.3, min_samples=10).fit(tfidf_matrix)')
/tmp/ipykernel_34186/629820274.py:24: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython

(24,)

In [17]:
# print(clusters)
print (mbk.labels_)

[ 1  4  6  5 19 14  3 18 17 10  8  0  2  3 14  0  8  2  5  3  0  8 13  2]


In [24]:
#k-means
clusterkm = km.labels_.tolist()

# #minikmeans
clustermbk = mbk.labels_.tolist()

#dbscan
clusters3 = labels

 #agglo
clusters4 = answer.tolist()

frame = pd.DataFrame(final_result["clear_item_name"], index = [clusterkm])

# #k-means
out = { 'title': range(len(result.index)), 'cluster': clusterkm }
frame1 = pd.DataFrame(out, index = [clusterkm], columns = ['title', 'cluster'])
frame1 = frame1.reset_index(drop=True)

#mini
out = { 'title': final_result["clear_item_name"], 'cluster': clustermbk }
frame_minik = pd.DataFrame(out, index = [clustermbk], columns = ['title', 'cluster'])

frame1['cluster'].value_counts()

dict = {}

for i in range(len(frame_minik.index)):
    dict[i] = frame1['cluster'][i]   

dict = pd.Series(dict)
dict

final = pd.concat([final_result, dict], ignore_index= True, axis=1)

# final = final.rename(columns = {0: "item_id", 1: "categories", 2: "composition", 3: "nutriotion", 4: "clear_name", 5: "cluster"})

# result
# frame_minik['cluster'].value_counts()
# final = final.sort_values(by='cluster')

In [25]:
final

,0,1,2,3
0,43499,Фарш из кролика и индейки,фарш кролика индейки,15
1,61936,"Фарш из мяса курицы, индейки и кролика",фарш мяса курицы индейки кролика,4
2,62798,"Фарш из говядины \Постный по стандарту ВВ\, 360 г",фарш говядины постный,12
3,62804,"Фарш из телятины, 360 г",фарш телятины,6
4,62806,"Фарш из говядины и свинины, 360 г",фарш говядины свинины,8
5,62809,"Фарш из говядины \по стандарту ВкусВилл\, 360 г",фарш говядины,1
6,69576,"Фарш \Домашний по стандарту ВкусВилл\, 360 г",фарш домашний,3
7,71029,Фарш из говядины и курицы,фарш говядины курицы,10
8,20336,"Фарш из кролика, 500 г",фарш кролика,14
9,21194,"Фарш куриный охлажденный, 500 г",фарш куриный охлажденный,13
